In [8]:
# Performance Benchmarking Notebook

import sys
sys.path.append('../src')

import time
import numpy as np
import torch
from data_generation.monte_carlo import generate_geometric_brownian_motion, european_option_price
from models.neural_network import OptionPricingNN

# Load trained model
model = OptionPricingNN(input_dim=5)
model.load_state_dict(torch.load('../data/models/option_pricing_nn.pth'))
model.eval()

# Benchmark parameters
n_prices = 1000
S0 = 100
r = 0.05
sigma = 0.2
T = 1.0
K = 100
n_paths = 10000
steps = 252

# Benchmark Monte Carlo
mc_times = []
for _ in range(5):  # Run multiple times for stable measurement
    start_time = time.time()

    for _ in range(n_prices):
        paths = generate_geometric_brownian_motion(S0, r, sigma, T, steps, n_paths)
        price = european_option_price(paths, K, r, T)

    mc_times.append(time.time() - start_time)

avg_mc_time = np.mean(mc_times)
print(f"Monte Carlo average time for {n_prices} prices: {avg_mc_time:.4f}s")

# Benchmark Neural Network
nn_times = []
inputs = torch.tensor([S0, K, T, r, sigma], dtype=torch.float32).unsqueeze(0)

for _ in range(5):  # Run multiple times for stable measurement
    start_time = time.time()

    with torch.no_grad():
        for _ in range(n_prices):
            price = model(inputs)

    nn_times.append(time.time() - start_time)

avg_nn_time = np.mean(nn_times)
print(f"Neural Network average time for {n_prices} prices: {avg_nn_time:.6f}s")

# Calculate speedup
speedup = avg_mc_time / avg_nn_time
print(f"Speedup: {speedup:.1f}x")

# Compare prices
paths = generate_geometric_brownian_motion(S0, r, sigma, T, steps, n_paths*10)  # More paths for accuracy
mc_price = european_option_price(paths, K, r, T)

with torch.no_grad():
    nn_price = model(inputs)

print(f"Monte Carlo price: {mc_price:.4f}")
print(f"Neural Network price: {nn_price.item():.4f}")
print(f"Percentage error: {abs(mc_price - nn_price.item())/mc_price*100:.2f}%")

# Save results
results = {
    'mc_time': avg_mc_time,
    'nn_time': avg_nn_time,
    'speedup': speedup,
    'mc_price': mc_price,
    'nn_price': nn_price.item(),
    'error_percentage': abs(mc_price - nn_price.item())/mc_price*100
}

np.save('../results/benchmark_results.npy', results)

print("Performance benchmarking complete!")

Monte Carlo average time for 1000 prices: 59.9105s
Neural Network average time for 1000 prices: 0.030252s
Speedup: 1980.4x
Monte Carlo price: 10.4649
Neural Network price: 9.7855
Percentage error: 6.49%
Performance benchmarking complete!
